In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
from pandas import ExcelWriter
from pandas import ExcelFile
from dateutil.parser import parse 
import re

In [2]:
simplyhired =pd.read_csv("file:///Users/manha/Desktop/DSE_6000/Active_Class/Project/Simplyhired/simplyhired_data_scientist.csv")

In [3]:
print(simplyhired.shape)
simplyhired.info()


(4740, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4740 entries, 0 to 4739
Data columns (total 8 columns):
Job_Title           4740 non-null object
Company_Name        4736 non-null object
Location            4740 non-null object
Date                4740 non-null object
Job_Description     4736 non-null object
Education           4740 non-null object
Skill               4740 non-null object
Estimated_Salary    4739 non-null object
dtypes: object(8)
memory usage: 296.3+ KB


In [4]:
#dropping any duplicate rows:
simplyhired = simplyhired.drop_duplicates()
simplyhired.reset_index(drop=True, inplace=True)
simplyhired.shape

(1314, 8)

In [5]:
simplyhired.isnull().values.any()
simplyhired.isnull().sum()

Job_Title           0
Company_Name        4
Location            0
Date                0
Job_Description     4
Education           0
Skill               0
Estimated_Salary    1
dtype: int64

In [6]:
#simplyhired['Estimated_Salary'].value_counts()

In [7]:
#some targeted cleaning of salary information to make parsing easier #—> remove “\n”, “$”, and “,”
simplyhired['Estimated_Salary'] = simplyhired['Estimated_Salary'].str.replace('\n', '')
simplyhired['Estimated_Salary'] = simplyhired['Estimated_Salary'].str.replace(',', '')
simplyhired['Estimated_Salary']= simplyhired['Estimated_Salary'].str.replace('$', '')
simplyhired['Estimated_Salary']= simplyhired['Estimated_Salary'].str.replace('Estimated:', '')
#simplyhired.tail()

In [8]:
sim_hour_salaries = simplyhired[simplyhired["Estimated_Salary"].str.contains('hour',na=False)]

simplyhired['salary_period'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
simplyhired.loc[simplyhired['Estimated_Salary'].str.contains('year',na=False), 'salary_period'] ='year'
simplyhired.loc[simplyhired['Estimated_Salary'].str.contains('month',na=False), 'salary_period'] ='month'
simplyhired.loc[simplyhired['Estimated_Salary'].str.contains('week',na=False), 'salary_period'] = 'week'
simplyhired.loc[simplyhired['Estimated_Salary'].str.contains('day',na=False), 'salary_period'] = 'day'
simplyhired.loc[simplyhired['Estimated_Salary'].str.contains('hour', na=False), 'salary_period'] ='hour'
simplyhired.loc[simplyhired['salary_period'].str.contains('NaN',na=True), 'salary_period'] ='Nothing_found'
#remove extra information from salary data
simplyhired['Estimated_Salary']= simplyhired['Estimated_Salary'].str.replace('year', '')
simplyhired['Estimated_Salary']= simplyhired['Estimated_Salary'].str.replace('a', '')
#simplyhired['Estimated_Salary']= simplyhired['Estimated_Salary'].str.replace('hour', '')
#simplyhired['Estimated_Salary']= simplyhired['Estimated_Salary'].str.replace('n hour', '')
#print(hour_salaries)
#simplyhired.tail()

In [9]:
#hour_salaries = simplyhired[simplyhired["Estimated_Salary"].str.contains('hour',na=False)]
#print(hour_salaries)
sim_hour_salaries["Estimated_Salary"] =sim_hour_salaries["Estimated_Salary"].str.replace('an hour', '')
print(sim_hour_salaries["Estimated_Salary"])


#simplyhired['Estimated_Salary'].value_counts()

15       55 - 75 
25            42 
28       60 - 65 
29       50 - 80 
43     100 - 150 
46       50 - 60 
52       65 - 75 
55            30 
57       33 - 46 
79       60 - 70 
133           85 
277      50 - 52 
280      28 - 34 
324      40 - 50 
330      20 - 60 
376      23 - 34 
589           48 
762      46 - 48 
813      60 - 75 
843           37 
Name: Estimated_Salary, dtype: object


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def split_sal(i):
    #print(i)
    #type(i)
    try:
        splt = i.split('-',1)
        #print(splt)
        first = float(splt[0])
        #print(first)
        second = float(splt[1])
        #print(second)
        return (first + second)/2
    except:
        return float(i)

sim_hour_salaries['Estimated_Salary'] =sim_hour_salaries['Estimated_Salary'].apply(split_sal)
#print(sim_hour_salaries['Estimated_Salary'] )

sim_hour_salaries['Estimated_Salary']=sim_hour_salaries['Estimated_Salary'] * 2080
#print(sim_hour_salaries['Estimated_Salary'])

#hour_salaries.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:

#rejoining salary data into main scrape_data df
combined_salaries = pd.concat([sim_hour_salaries], axis=0, sort=False)
sf = pd.concat([simplyhired, combined_salaries], axis=0, sort= False)
sf.head()

,Job_Title,Company_Name,Location,Date,Job_Description,Education,Skill,Estimated_Salary,salary_period
0,Data Scientist,Degreed,Remote,"<time datetime=""2019-09-24T02:26:53Z"" itemprop...",SkillsSQLCommunication SkillsClusteringNatural...,SQLCommunication SkillsClusteringNatural Langu...,SkillsSQLCommunication SkillsClusteringNatural...,110000 - 140000,year
1,Entry Level Data Scientist,IBM,United States,"<time datetime=""2019-08-03T10:12:38Z"" itemprop...",EducationMaster's DegreeSkillsDoD ExperienceSu...,Master's Degree,EducationMaster's DegreeSkillsDoD ExperienceSu...,94000 - 130000,year
2,Data Scientist - Nationwide Opportunities,"Amazon Web Services, Inc.",Florida +42 locations,"<time datetime=""2019-06-21T07:53:00Z"" itemprop...",EducationBachelor's DegreeSkillsCommunication ...,Bachelor's Degree,EducationBachelor's DegreeSkillsCommunication ...,91000 - 130000,year
3,Data Scientist - Entry Level,Numerdox,"Sacramento, CA","<time datetime=""2018-11-15T08:11:28Z"" itemprop...",SkillsSQL401(k)HadoopAWSSparkBenefitsFlexible ...,SQL401(k)HadoopAWSSpark,SkillsSQL401(k)HadoopAWSSparkBenefitsFlexible ...,110000 - 160000,year
4,Data Scientist,Quaxigma,"Schaumburg, IL","<time datetime=""2019-08-18T20:12:53Z"" itemprop...",EducationMaster's DegreeDoctoral DegreeSkillsS...,Master's DegreeDoctoral Degree,EducationMaster's DegreeDoctoral DegreeSkillsS...,96000 - 130000,year


In [12]:
sf['Estimated_Salary']= sf['Estimated_Salary'].str.replace('n hour', '')

In [13]:
def split_sal(i):
    #print(i)
    #type(i)
    try:
        splt = i.split('-',1)
        #print(splt)
        first = float(splt[0])
        #print(first)
        second = float(splt[1])
        #print(second)
        return (first + second)/2
    except:
        return float(i)

sf['Estimated_Salary'] = sf['Estimated_Salary'].apply(split_sal)

In [14]:
#sf.head()

In [15]:
#sf['Estimated_Salary'].value_counts()
# Get names of indexes for which column Age has value 30
indexNames = sf[ sf['Estimated_Salary'] <= 300 ].index
print(indexNames)
 
# Delete these row indexes from dataFrame
sf.drop(indexNames , inplace=True)

Int64Index([ 15,  25,  28,  29,  43,  46,  52,  55,  57,  79, 133, 277, 280,
            324, 330, 376, 589, 762, 813, 843],
           dtype='int64')


In [16]:
#sf['Estimated_Salary'].value_counts()

In [17]:
sf.drop('Education', axis=1, inplace=True)
sf.drop('Skill', axis=1, inplace=True)
sf.drop('Date', axis=1, inplace=True)
sf.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period
0,Data Scientist,Degreed,Remote,SkillsSQLCommunication SkillsClusteringNatural...,125000.0,year
1,Entry Level Data Scientist,IBM,United States,EducationMaster's DegreeSkillsDoD ExperienceSu...,112000.0,year
2,Data Scientist - Nationwide Opportunities,"Amazon Web Services, Inc.",Florida +42 locations,EducationBachelor's DegreeSkillsCommunication ...,110500.0,year
3,Data Scientist - Entry Level,Numerdox,"Sacramento, CA",SkillsSQL401(k)HadoopAWSSparkBenefitsFlexible ...,135000.0,year
4,Data Scientist,Quaxigma,"Schaumburg, IL",EducationMaster's DegreeDoctoral DegreeSkillsS...,113000.0,year


In [18]:
#creating data character cleaning function, and applying to all 
#columns, also lowercasing all string data for ease of later nlp
def data_clean(df, column):
    cleaning_list = ['+', '$','/',',','?','.',';','-','@','!','&','%','^','*',')','(', '\n']
    for item in cleaning_list:
        df[column] = df[column].str.replace(item, '')
##can’t clean the salary column due to float values, and don’t need 
#to clean og_salary, so keeping out of the for loop
for column in simplyhired.columns[0:len(simplyhired.columns)-2]:
    data_clean(simplyhired, column)
#indeed.head()


In [19]:
sf['Location'].value_counts()

Atlanta, GA                     82
Chicago, IL                     75
San Francisco, CA               58
New York, NY                    54
Livermore, CA                   52
Rolling Meadows, IL             41
San Jose, CA                    39
United States                   34
Washington, DC                  31
Boston, MA                      28
Seattle, WA                     23
Tampa, FL                       22
Sunnyvale, CA                   21
Marshfield, WI                  19
Lake Mary, FL                   19
Amherst, MA                     16
Menlo Park, CA                  15
Austin, TX                      13
Pittsburgh, PA                  13
Denver, CO                      13
Arlington, VA                   11
Houston, TX                     11
Pasadena, CA                    11
Cincinnati, OH                  10
Remote                           8
Palo Alto, CA                    7
McLean, VA                       7
San Diego, CA                    7
Reston, VA          

In [20]:
#creating data character cleaning function, and applying to all 

cleaning_list = ['1 location ',' 2 locations ' ]
for item in cleaning_list:
    sf["Location"] = sf["Location"].str.replace(item,'' )

#indeed.head()
#sf['Location'].value_counts()

In [21]:
sf['State'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
sf.loc[sf['Location'].str.contains('PA',na=False), 'State'] ='PA'
sf.loc[sf['Location'].str.contains('MA',na=False), 'State'] ='MA'
sf.loc[sf['Location'].str.contains('TX',na=False), 'State'] ='TX'
sf.loc[sf['Location'].str.contains('FL',na=False), 'State'] ='FL'
sf.loc[sf['Location'].str.contains('MO',na=False), 'State'] ='MO'
sf.loc[sf['Location'].str.contains('IN',na=False), 'State'] ='IN'
sf.loc[sf['Location'].str.contains('MN',na=False), 'State'] ='MN'
sf.loc[sf['Location'].str.contains('CO',na=False), 'State'] ='CO'
sf.loc[sf['Location'].str.contains('FL',na=False), 'State'] ='DC'
sf.loc[sf['Location'].str.contains('NJ',na=False), 'State'] ='NJ'
sf.loc[sf['Location'].str.contains('AZ',na=False), 'State'] ='AZ'
sf.loc[sf['Location'].str.contains('MI',na=False), 'State'] ='MI'
sf.loc[sf['Location'].str.contains('MD',na=False), 'State'] ='MD'
sf.loc[sf['Location'].str.contains('WA',na=False), 'State'] ='WA'
sf.loc[sf['Location'].str.contains('TN',na=False), 'State'] ='TN'
sf.loc[sf['Location'].str.contains('SD',na=False), 'State'] ='SD'
sf.loc[sf['Location'].str.contains('CT',na=False), 'State'] ='CT'
sf.loc[sf['Location'].str.contains('VA',na=False), 'State'] ='VA'
sf.loc[sf['Location'].str.contains('OH',na=False), 'State'] ='OH'
sf.loc[sf['Location'].str.contains('WI',na=False), 'State'] ='WI'
sf.loc[sf['Location'].str.contains('NY',na=False), 'State'] ='NY'
sf.loc[sf['Location'].str.contains('IL',na=False), 'State'] ='IL'
sf.loc[sf['Location'].str.contains('OR',na=False), 'State'] ='OR'
sf.loc[sf['Location'].str.contains('IA',na=False), 'State'] ='IA'
sf.loc[sf['Location'].str.contains('WV',na=False), 'State'] ='WV'
sf.loc[sf['Location'].str.contains('IN',na=False), 'State'] ='IN'
sf.loc[sf['Location'].str.contains('CA',na=False), 'State'] ='CA'
sf.loc[sf['Location'].str.contains('United States',na=False), 'State'] ='United States'
sf.loc[sf['Location'].str.contains('Colorado',na=False), 'State'] ='CO'
sf.loc[sf['State'].str.contains('NaN',na=True), 'State'] ='Not Mention'
#simplyhired.head()
#sf['Location'].value_counts()

In [22]:
sf['State'].value_counts()

CA               296
Not Mention      201
IL               142
VA                80
NY                70
DC                66
MA                62
TX                53
MD                41
United States     36
WA                33
OH                33
MI                31
CO                29
WI                25
PA                25
NJ                17
IN                 9
CT                 9
MO                 8
OR                 6
TN                 6
MN                 5
AZ                 4
SD                 3
WV                 2
IA                 2
Name: State, dtype: int64

In [23]:
simplyhired['City'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
sf.loc[sf['Location'].str.contains('Atlanta',na=False), 'City'] ='Atlanta'
sf.loc[sf['Location'].str.contains('Chicago',na=False), 'City'] ='Chicago'
sf.loc[sf['Location'].str.contains('San Francisco',na=False), 'City'] ='San Francisco'
sf.loc[sf['Location'].str.contains('New York',na=False), 'City'] ='New York'
sf.loc[sf['Location'].str.contains('Livermore',na=False), 'City'] ='Livermore'
sf.loc[sf['Location'].str.contains('Redwood City',na=False), 'City'] ='Redwood City'
sf.loc[sf['Location'].str.contains('Maplewood',na=False), 'City'] ='Maplewood'
sf.loc[sf['Location'].str.contains('Indianapolis',na=False), 'City'] ='Indianapolis'
sf.loc[sf['Location'].str.contains('Colorado',na=False), 'City'] ='Colorado'
sf.loc[sf['Location'].str.contains('Oshkosh',na=False), 'City'] ='Oshkosh'
sf.loc[sf['Location'].str.contains('Memphis',na=False), 'City'] ='Memphis'
sf.loc[sf['Location'].str.contains('Delafield',na=False), 'City'] ='Delafield'
sf.loc[sf['Location'].str.contains('Woodcliff',na=False), 'City'] ='Woodcliff'
sf.loc[sf['Location'].str.contains('Tempe',na=False), 'City'] ='Tempe'
sf.loc[sf['Location'].str.contains('Morrisville',na=False), 'City'] ='Morrisville'
sf.loc[sf['Location'].str.contains('Fishers',na=False), 'City'] ='Fishers'
sf.loc[sf['Location'].str.contains('Chester',na=False), 'City'] ='Chester'
sf.loc[sf['Location'].str.contains('Johnston',na=False), 'City'] ='Johnston'
sf.loc[sf['Location'].str.contains('Aguadilla',na=False), 'City'] ='Aguadilla'
sf.loc[sf['Location'].str.contains('Brookfield',na=False), 'City'] ='Brookfield'
sf.loc[sf['Location'].str.contains('Malibu',na=False), 'City'] ='Malibu'
sf.loc[sf['Location'].str.contains('Bridgeport',na=False), 'City'] ='Bridgeport'
sf.loc[sf['Location'].str.contains('Ada',na=False), 'City'] ='Ada'
sf.loc[sf['Location'].str.contains('Urbandale',na=False), 'City'] ='Urbandale'
sf.loc[sf['Location'].str.contains('Englewood',na=False), 'City'] ='Englewood'
sf.loc[sf['Location'].str.contains('Quantico',na=False), 'City'] ='Quantico'
sf.loc[sf['Location'].str.contains('Albany',na=False), 'City'] ='Albany'
sf.loc[sf['Location'].str.contains('Chantilly',na=False), 'City'] ='Chantilly'
sf.loc[sf['Location'].str.contains('Alpharetta',na=False), 'City'] ='Alpharetta'
sf.loc[sf['Location'].str.contains('Culver City',na=False), 'City'] ='Culver City'
sf.loc[sf['Location'].str.contains('Hartford',na=False), 'City'] ='Hartford'
sf.loc[sf['Location'].str.contains('Bethesda',na=False), 'City'] ='Bethesda'
sf.loc[sf['Location'].str.contains('Reston',na=False), 'City'] ='Reston'
sf.loc[sf['Location'].str.contains('San Diego',na=False), 'City'] ='San Diego'
sf.loc[sf['Location'].str.contains('Ann Arbor',na=False), 'City'] ='Ann Arbor'
sf.loc[sf['Location'].str.contains('McLean',na=False), 'City'] ='McLean'
sf.loc[sf['Location'].str.contains('Remote',na=False), 'City'] ='Remote'
sf.loc[sf['Location'].str.contains('Cincinnati',na=False), 'City'] ='Cincinnati'
sf.loc[sf['Location'].str.contains('Arlington',na=False), 'City'] ='Arlington'
sf.loc[sf['Location'].str.contains('Houston',na=False), 'City'] ='Houston'
sf.loc[sf['Location'].str.contains('Pasadena',na=False), 'City'] ='Pasadena'
sf.loc[sf['Location'].str.contains('Austin',na=False), 'City'] ='Austin'
sf.loc[sf['Location'].str.contains('Denver',na=False), 'City'] ='Denver'
sf.loc[sf['Location'].str.contains('Menlo Park',na=False), 'City'] ='Menlo Park'
sf.loc[sf['Location'].str.contains('Pittsburgh',na=False), 'City'] ='Pittsburgh'
sf.loc[sf['Location'].str.contains('Amherst',na=False), 'City'] ='Amherst'
sf.loc[sf['Location'].str.contains('Marshfield',na=False), 'City'] ='Marshfield'
sf.loc[sf['Location'].str.contains('Lake Mary',na=False), 'City'] ='Lake Mary'
sf.loc[sf['Location'].str.contains('Sunnyvale',na=False), 'City'] ='Sunnyvale'
sf.loc[sf['Location'].str.contains('Tampa',na=False), 'City'] ='Tampa'
sf.loc[sf['Location'].str.contains('Seattle',na=False), 'City'] ='Seattle'
sf.loc[sf['Location'].str.contains('Boston',na=False), 'City'] ='Boston'
sf.loc[sf['Location'].str.contains('Washington',na=False), 'City'] ='Washington'
sf.loc[sf['Location'].str.contains('San Jose',na=False), 'City'] ='San Jose'
sf.loc[sf['Location'].str.contains('Rolling Meadows',na=False), 'City'] ='Rolling Meadows'
sf.loc[sf['Location'].str.contains('Reston',na=False), 'City'] ='Reston'
sf.loc[sf['Location'].str.contains('San Diego',na=False), 'City'] ='San Diego'
sf.loc[sf['Location'].str.contains('Ann Arbor',na=False), 'City'] ='Ann Arbor'
sf.loc[sf['Location'].str.contains('McLean',na=False), 'City'] ='McLean'
sf.loc[sf['Location'].str.contains('Remote',na=False), 'City'] ='Remote'
sf.loc[sf['Location'].str.contains('Cincinnati',na=False), 'City'] ='Cincinnati'
sf.loc[sf['Location'].str.contains('Arlington',na=False), 'City'] ='Arlington'
sf.loc[sf['Location'].str.contains('Houston',na=False), 'City'] ='Houston'
sf.loc[sf['Location'].str.contains('Pasadena',na=False), 'City'] ='Pasadena'
sf.loc[sf['Location'].str.contains('Austin',na=False), 'City'] ='Austin'
sf.loc[sf['Location'].str.contains('Denver',na=False), 'City'] ='Denver'
sf.loc[sf['Location'].str.contains('United States',na=False), 'City'] ='United States'
sf.loc[sf['City'].str.contains('NaN',na=True), 'City'] ='Not Mention'


sf.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City
0,Data Scientist,Degreed,Remote,SkillsSQLCommunication SkillsClusteringNatural...,125000.0,year,Not Mention,Remote
1,Entry Level Data Scientist,IBM,United States,EducationMaster's DegreeSkillsDoD ExperienceSu...,112000.0,year,United States,United States
2,Data Scientist - Nationwide Opportunities,"Amazon Web Services, Inc.",Florida +42 locations,EducationBachelor's DegreeSkillsCommunication ...,110500.0,year,Not Mention,Not Mention
3,Data Scientist - Entry Level,Numerdox,"Sacramento, CA",SkillsSQL401(k)HadoopAWSSparkBenefitsFlexible ...,135000.0,year,CA,Not Mention
4,Data Scientist,Quaxigma,"Schaumburg, IL",EducationMaster's DegreeDoctoral DegreeSkillsS...,113000.0,year,IL,Not Mention


In [24]:
sf['Python'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
sf.loc[sf['Job_Description'].str.contains('Python',na=False), 'Python'] ='1'
sf.loc[sf['Job_Description'].str.contains('python',na=False), 'Python'] ='1'
sf.loc[sf['Python'].str.contains('NaN',na=True), 'Python'] ='0'
sf['SQL'] = np.nan
sf.loc[sf['Job_Description'].str.contains('SQL',na=False), 'SQL'] ='1'
sf.loc[sf['Job_Description'].str.contains('sql',na=False), 'SQL'] ='1'
sf.loc[sf['SQL'].str.contains('NaN',na=True), 'SQL'] ='0'
sf['NoSQL'] = np.nan
sf.loc[sf['Job_Description'].str.contains('NoSQL',na=False), 'NoSQL'] ='1'
sf.loc[sf['NoSQL'].str.contains('NaN',na=True), 'NoSQL'] ='0'
sf['java'] = np.nan
sf.loc[sf['Job_Description'].str.contains('java',na=False), 'java'] ='1'
sf.loc[sf['Job_Description'].str.contains('JavaScript',na=False), 'java'] ='1'
sf.loc[sf['java'].str.contains('NaN',na=True), 'java'] ='0'
sf.head()

,Job_Title,Company_Name,Location,Job_Description,Estimated_Salary,salary_period,State,City,Python,SQL,NoSQL,java
0,Data Scientist,Degreed,Remote,SkillsSQLCommunication SkillsClusteringNatural...,125000.0,year,Not Mention,Remote,1,1,0,0
1,Entry Level Data Scientist,IBM,United States,EducationMaster's DegreeSkillsDoD ExperienceSu...,112000.0,year,United States,United States,0,0,0,0
2,Data Scientist - Nationwide Opportunities,"Amazon Web Services, Inc.",Florida +42 locations,EducationBachelor's DegreeSkillsCommunication ...,110500.0,year,Not Mention,Not Mention,0,0,0,0
3,Data Scientist - Entry Level,Numerdox,"Sacramento, CA",SkillsSQL401(k)HadoopAWSSparkBenefitsFlexible ...,135000.0,year,CA,Not Mention,1,1,0,0
4,Data Scientist,Quaxigma,"Schaumburg, IL",EducationMaster's DegreeDoctoral DegreeSkillsS...,113000.0,year,IL,Not Mention,1,1,0,1


In [25]:
sf['swift'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Swift',na=False), 'swift'] ='1'
sf.loc[sf['swift'].str.contains('NaN',na=True), 'swift'] ='0'
sf['PHP'] = np.nan
sf.loc[sf['Job_Description'].str.contains('PHP',na=False), 'PHP'] ='1'
sf.loc[sf['PHP'].str.contains('NaN',na=True), 'PHP'] ='0'

sf['R'] = np.nan
sf.loc[sf['Job_Description'].str.contains('R',na=False), 'R'] ='1'
sf.loc[sf['R'].str.contains('NaN',na=True), 'R'] ='0'
#indeed.head()

#indeed.head(100)

In [26]:
sf['Project_Management'] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
#def main():
   # foo = """Project Management"""
sf.loc[sf['Job_Description'].str.contains('Project Management',na=False), 'Project_Management'] ='1'
sf.loc[sf['Job_Description'].str.contains('ProjectManagement',na=False), 'Project_Management'] ='1'
sf.loc[sf['Job_Description'].str.contains('Management',na=False), 'Project_Management'] ='1'
sf.loc[sf['Project_Management'].str.contains('NaN',na=True), 'Project_Management'] ='0'
sf['Risk_Management'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Risk Management',na=False), 'Risk_Management'] ='1'
sf.loc[sf['Job_Description'].str.contains('RiskManagement',na=False), 'Risk_Management'] ='1'
sf.loc[sf['Risk_Management'].str.contains('Nan',na=True), 'Risk_Management'] ='0'
sf['Analysis_Skill'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Analysis Skill',na=False), 'Analysis_Skill'] ='1'
sf.loc[sf['Job_Description'].str.contains('AnalysisSkill',na=False), 'Analysis_Skill'] ='1'
sf.loc[sf['Job_Description'].str.contains('Analysis',na=False), 'Analysis_Skill'] ='1'
sf.loc[sf['Analysis_Skill'].str.contains('NaN',na=True), 'Analysis_Skill'] ='0'
sf['Hadoop'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Hadoop',na=False), 'Hadoop'] ='1'
sf.loc[sf['Hadoop'].str.contains('NaN',na=True), 'Hadoop'] ='0'
sf['SAS'] = np.nan
sf.loc[sf['Job_Description'].str.contains('SAS',na=False), 'SAS'] ='1'
sf.loc[sf['SAS'].str.contains('NaN',na=True), 'SAS'] ='0'
sf['Microsoft_Word'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Microsoft Word',na=False), 'Microsoft_Word'] ='1'
sf.loc[sf['Microsoft_Word'].str.contains('NaN',na=True), 'Microsoft_Word'] ='0'
sf['Microsoft_Office'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Microsoft Office',na=False), 'Microsoft_Office'] ='1'
sf.loc[sf['Job_Description'].str.contains('Microsoft office',na=False), 'Microsoft_Office'] ='1'                                                 
sf.loc[sf['Microsoft_Office'].str.contains('NaN',na=True), 'Microsoft_Office'] = '0'
           

#indeed.head()

In [27]:
sf['Hive'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Hive',na=False), 'Hive'] ='1'
sf.loc[sf['Hive'].str.contains('NaN',na=True), 'Hive'] ='0'
sf['AWS'] = np.nan
sf.loc[sf['Job_Description'].str.contains('AWS',na=False), 'AWS'] ='1'
sf.loc[sf['AWS'].str.contains('NaN',na=True), 'AWS'] ='0'
sf['Spark'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Spark',na=False), 'Spark'] ='1'
sf.loc[sf['Spark'].str.contains('NaN',na=True), 'Spark'] ='0'
sf['Data_Mining'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Data Mining',na=False), 'Data_Mining'] ='1'
sf.loc[sf['Job_Description'].str.contains('DataMining',na=False), 'Data_Mining'] ='1'
sf.loc[sf['Data_Mining'].str.contains('NaN',na=True), 'Data_Mining'] ='0'
sf['Predictive_Analytics'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Predictive Analytics',na=False), 'Predictive_Analytics'] ='1'
sf.loc[sf['Job_Description'].str.contains('PredictiveAnalytics',na=False), 'Predictive_Analytics'] ='1'
sf.loc[sf['Predictive_Analytics'].str.contains('NaN',na=True), 'Predictive_Analytics'] ='0'
sf['Machine_Learning'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Machine Learning',na=False), 'Machine_Learning'] ='1'
sf.loc[sf['Job_Description'].str.contains('MachineLearning',na=False), 'Machine_Learning'] ='1'

sf.loc[sf['Machine_Learning'].str.contains('NaN',na=True), 'Machine_Learning'] ='0'

#indeed.head()

In [28]:
sf['Pig'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Pig',na=False), 'Pig'] ='1'
sf.loc[sf['Pig'].str.contains('NaN',na=True), 'Pig'] ='0'
sf['Data_Analysis_Skill'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Data Analysis Skill',na=False), 'Data_Analysis_Skill'] ='1'
sf.loc[sf['Job_Description'].str.contains('DataAnalysisSkill',na=False), 'Data_Analysis_Skill'] ='1'
sf.loc[sf['Job_Description'].str.contains('Data Analysis',na=False), 'Data_Analysis_Skill'] ='1'
sf.loc[sf['Job_Description'].str.contains('DataAnalysis',na=False), 'Data_Analysis_Skill'] ='1'
sf.loc[sf['Data_Analysis_Skill'].str.contains('NaN',na=True), 'Data_Analysis_Skill'] ='0'
sf['Communication_Skill'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Communication Skill',na=False), 'Communication_Skill'] ='1'
sf.loc[sf['Job_Description'].str.contains('CommunicationSkill',na=False), 'Communication_Skill'] ='1'

sf.loc[sf['Communication_Skill'].str.contains('NaN',na=True), 'Communication_Skill'] ='0'

In [29]:
sf.loc[sf['Job_Description'].str.contains('matlab',na=False), 'Matlab'] ='1'
sf.loc[sf['Matlab'].str.contains('NaN',na=True), 'Matlab'] ='0'
sf['Scala'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Scala',na=False), 'Scala'] ='1'
sf.loc[sf['Scala'].str.contains('NaN',na=True), 'Scala'] ='0'

In [30]:
#Education
sf['Bachelor_Degree'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Bachelor',na=False), 'Bachelor_Degree'] ='1'
sf.loc[sf['Bachelor_Degree'].str.contains('NaN',na=True), 'Bachelor_Degree'] ='0'
sf['Master_Degree'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Master Degree',na=False), 'Master_Degree'] ='1'
sf.loc[sf['Job_Description'].str.contains('Master',na=False), 'Master_Degree'] ='1'
sf.loc[sf['Master_Degree'].str.contains('NaN',na=True), 'Master_Degree'] ='0'

sf['Diploma_GED'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Diploma',na=False), 'Diploma_GED'] ='1'
sf.loc[sf['Job_Description'].str.contains('GED',na=False), 'Diploma_GED'] ='1'

sf.loc[sf['Diploma_GED'].str.contains('NaN',na=True), 'Diploma_GED'] ='0'

sf['Doctoral_Degree'] = np.nan
sf.loc[sf['Job_Description'].str.contains('Doctoral',na=False), 'Doctoral_Degree'] ='1'
sf.loc[sf['Doctoral_Degree'].str.contains('NaN',na=True), 'Doctoral_Degree'] ='0'



And that’s it! The final step was to save my data as a cleaned csv to make it easier to load and begin my modeling.

In [31]:
sf.to_csv('sf.csv')

In [32]:
#sf.info()
print(sf.shape)

(1294, 37)
